## QA RAG

https://www.youtube.com/watch?v=jSd6pdydBNU&list=PL5bzmUGXvZNRPQi4PAGFkA2inN-tRF4Mi&index=13 

위 영상을 참고하자 


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from pprint import pprint

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("http://www.infopub.co.kr/new/include/detail.asp?sku=05000274",),
)
docs = loader.load()

print(len(docs))
print('---'*10)
print(docs[0])
print('---'*10)
pprint(docs[0].metadata)

USER_AGENT environment variable not set, consider setting it to identify your requests.


1
------------------------------
page_content='\n\n\n\n\n[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드]\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n파이썬 머신러닝 판다스 데이터 분석 개정판\r\n\n\n\n저자:\n\n오승환\n\n\n\n역자:\n\n\n\n\n구분:\n\r\n\t\r\n\t\r\n       국내서\r\n    \r\n\t\r\n\t\n\n\n발행일:\n2024년 6월 25일\n\n\n정가:\n30,000원\n\n\n페이지:\n584 페이지\n\n\nISBN:\n978-89-5674-980-8\n\n\n출판사:\n\n\r\n\t\t\t\t정보문화사\r\n\r\n\t\n\n\n판형:\n187×235\n\n\n\n난이도:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n책소개\n저자소개\n미리보기\n오탈자\n보도자료실\n샘플자료 다운로드\n\n\n\n데이터 과학자가 되기 위한 첫걸음!어려운 이론은 최소화하고, 예제 코드를 따라 하며 자연스럽게 사용법에 익숙해지도록 안내하는 데이터 분석 입문서다. 데이터 분석을 처음 배우는 입문자의 입장에서 고급 이론과 데이터 분석 도구를 함께 배우는 것의 어려움을 아는 저자가, 데이터 분석에 필요한 필수 라이브러리를 소개하고 설치부터 예제 코드까지 따라 할 수 있게 구성했다. 개념 이해를 돕기 위해 다이어그램 등 풍부한 도식화도 적극 활용했다. 1판에서 큰 인기를 얻어 준비된 이번 개정판에서는 저자가 실무에서 쌓은 경험을 자연

In [3]:
# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer

# 토크나이저 
tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# Token 수를 기준으로 문서를 청크 단위로 분할
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 120,           # 120개의 단어 크기로 청크
    chunk_overlap  = 10,        # 10단어는 중첩이 되도록 
)

split_docs = text_splitter.split_documents(docs)            # 문서를 위에서 설정한 text_splitter로 쪼개줌 
print(len(split_docs))
print(split_docs[0])

36
page_content='[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드]' metadata={'source': 'http://www.infopub.co.kr/new/include/detail.asp?sku=05000274', 'title': '[정보문화사] IT 도서의 새로운 패러다임', 'language': 'No language found.'}


In [4]:
# HugoingFace Embeddings를 다운로드
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from langchain_openai import ChatOpenAI

# 임베딩 모델 
embeddings_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",           # 한국어 fine-tuning한 임베딩 모델 
)


vectorstore = Chroma.from_documents(documents=split_docs,               # 자른 내용 
                                    embedding=embeddings_model)         # 임베딩모델로 임베딩하여 Chroma를 통해 vectorstore로 저장 

retriever = vectorstore.as_retriever()                                  # vectorstore 전용 검색기 

# 기본 질문 대답 Prompt 템플릿 설정 
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chat Model
# llm = ChatOpenAI(model='gpt-4o')
llm = ChatOllama(model="qwen2")
# llm = ChatOllama(model='gemma2')

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
response = rag_chain.invoke("이 책의 특징을 3가지 요점으로 설명해주세요.")
print(response)

c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


1. **새로운 버전의 Pandas 활용**: 이 책은 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하고 있습니다. 이를 통해 독자는 최신 기술과 방법에 대해 배우고 실력을 향상시킬 수 있습니다.

2. **시스템적 학습 구성**: 책의 구성이 시스템적이며, Pandas를 설치하기 위한 단계부터 시작하여 각각의 예제 코드까지 제공합니다. 이로써 독자는 직접적으로 따라 할 수 있으며, 실질적인 경험을 통해 배울 수 있습니다.

3. **개념 이해와 도시화**: 개념을 이해하는 데 도움이 되도록 다양한 다이어그램과 풍부한 도시화가 적극 활용됩니다. 이로써 독자는 복잡한 이론을 쉽게 이해하고, 이를 실무에 적용할 수 있습니다.


## Test Data 만들기 - AutoRAG 활용

https://github.com/Marker-Inc-Korea/AutoRAG

In [5]:
# !pip install AutoRAG

In [6]:
# 생성한 이벤트 루프와의 중첩을 피하기 위해 
import nest_asyncio
nest_asyncio.apply()

In [7]:
from autorag.data.corpus import langchain_documents_to_parquet
import os 
import pandas as pd 
from os import path

path = './corpus_data'

os.makedirs(path, exist_ok=True)
file = f'{path}/pandas_book.parquet'
if not os.path.exists(file):
    # text_splitter를 통해 36개의 청크로 잘라 놓은 split_docs를 .parquet 형태로 변환함 
    corpus_df = langchain_documents_to_parquet(split_docs, file)
    
else:
    corpus_df = pd.read_parquet(file)

c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
corpus_df

,doc_id,contents,metadata
0,d86f1049-c718-4953-a13a-36a2a89754c1,[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드],"{'language': 'No language found.', 'last_modif..."
1,d84efc62-cb6c-495e-af58-96049a1277fd,파이썬 머신러닝 판다스 데이터 분석 개정판\r\n\n\n\n저자:,"{'language': 'No language found.', 'last_modif..."
2,1f60783b-6e34-4fe5-8dd6-607aaca32eb0,저자:\n\n오승환\n\n\n\n역자:\n\n\n\n\n구분:\n\r\n\t\r\n...,"{'language': 'No language found.', 'last_modif..."
3,27478669-ce70-44d9-a5ad-37dd246e02e1,난이도:,"{'language': 'No language found.', 'last_modif..."
4,21ea171c-0c15-44b4-b4dd-acfeb63166e3,책소개\n저자소개\n미리보기\n오탈자\n보도자료실\n샘플자료 다운로드,"{'language': 'No language found.', 'last_modif..."
5,b6d369d2-63d8-4910-977c-beb1686e8fcd,"데이터 과학자가 되기 위한 첫걸음!어려운 이론은 최소화하고, 예제 코드를 따라 하며...","{'language': 'No language found.', 'last_modif..."
6,4778a047-b33b-40bf-ace3-5541d28ca54c,분석을 처음 배우는 입문자의 입장에서 고급 이론과 데이터 분석 도구를 함께 배우는 ...,"{'language': 'No language found.', 'last_modif..."
7,6db40309-0edc-4025-91b7-3b38bb31e3e6,소개하고 설치부터 예제 코드까지 따라 할 수 있게 구성했다. 개념 이해를 돕기 위해...,"{'language': 'No language found.', 'last_modif..."
8,5b79f309-73e6-4fed-b524-e3f4abfdd46c,큰 인기를 얻어 준비된 이번 개정판에서는 저자가 실무에서 쌓은 경험을 자연스럽게 녹...,"{'language': 'No language found.', 'last_modif..."
9,a2defa5b-8d29-4be9-9a1c-734ae9908da4,"만큼 더욱 자세한 설명과 다양한 데이터 시각화, 데이터 전처리 기법을 담았으며, P...","{'language': 'No language found.', 'last_modif..."


### AutoRAG를 통해 데이터로부터 질문과 대답 (라벨 데이터) 구축

https://ollama.com/library/gemma2

에서 9b 짜리 모델 다운 스크립트 복사하고 prompt에 붙여넣기 해서 모델 다운 받기 

너무 무겁다, 나는 데탑에서 돌렸는데, 아마 노트북에서는 안돌아 갈것이다. 

In [9]:
# # LangChain
# from langchain_community.llms import Ollama
# llm = Ollama(model="gemma2")
# llm.invoke("Why is the sky blue?")

In [10]:
# # LlamaIndex
# from llama_index.llms.ollama import Ollama
# llm = Ollama(model="gemma2")
# llm.complete("Why is the sky blue?")

In [11]:
import pandas as pd
from autorag.data.qacreation import generate_qa_llama_index, make_single_content_qa
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI


# 데이터로부터 스스로 질문과 대답을 만들어 낸다. <-AutoRag를 통해  
prompt = """
Generate question and answer pairs for the given passage.

Passage:
{{text}}

Number of questions to generate: {{num_questions}}

Example:
[Q]: 이 책의 글쓴이는 누구인가요?
[A]: 저자의 이름은 홍길동입니다.

Result:
"""

# 데이터 읽어오기 
corpus_df = pd.read_parquet(f'{path}/pandas_book.parquet')

# LLM모델 선언 구글의 gemma2가 6월 27일 발표된 모델임, Llama3-8b모델에 비해 Gemma2-9b모델이 성능이 좋음
# llm = Ollama(model='gemma2', temperature=1.0)     # 원래는 이거 쓰려고 했는데, 그냥 API 쓸수 있는 모델 쓰자 
# llm = Ollama(model="qwen2", temperature=1.0)      # 좋음 
llm = OpenAI(model='gpt-4o', temperature=1.0)


qa_df = make_single_content_qa(corpus_df,                                   # 전처리한 데이터 
                               content_size=len(corpus_df),                 # 데이터(행/청크) 개수 
                               qa_creation_func=generate_qa_llama_index,    # 질의 응답을 생성하는 함수 
                               llm=llm,                                     # LLM 모델 
                               prompt=prompt,                               # 질문과 대답 형식 프롬프트 
                               question_num_per_content=1)                  # 36개의 컨텐츠당 만들어 낼 질문-대답 수 (1이면 1:1비율로 만듬)


qa_df.head()

[06/29/24 22:11:14] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=202200;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=525928;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[06/29/24 22:11:15] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=842357;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=742881;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=508767;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=723903;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=712153;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=620035;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[06/29/24 22:11:16] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=582340;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=158961;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

[06/29/24 22:11:25] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=400643;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=90239;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[06/29/24 22:11:26] INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=669152;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=649086;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=272853;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=767043;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1773] >> HTTP Request: POST                                ]8;id=348270;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=848038;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\httpx\_client.py#1773\1773]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

,qid,retrieval_gt,query,generation_gt
0,98290e07-565e-4eda-b028-c8d28af7e589,[[f3b63111-8036-4f21-a629-667f3b416c72]],"""본문 소스 코드"" 자료의 등록일은 언제인가요?",[등록일은 2024년 6월 25일입니다.]
1,ff487150-3312-491d-8e4a-7869bc41e067,[[6efb506d-100e-4b81-be0e-cd39bbf237cb]],이 책에 추가된 주요 내용은 무엇인가요?,[시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.]
2,2b486627-da24-4d56-8abf-03b09ef9b642,[[70b74199-9865-4844-9355-8ed2de5febf8]],"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표일은 언제인가요?",[2024년 4월 24일입니다.]
3,680ab4e7-fa97-4167-98ec-6a874888c518,[[241cab43-3017-40bf-b703-12a6029a05ca]],[보도자료] 파이썬과 엑셀로 시작하는 딥러닝이 발표된 날짜는 언제인가요?,[이 보도자료는 2023년 7월 19일에 발표되었습니다.]
4,54b9798b-91aa-484c-8544-4600cbdabe38,[[039c58d5-6083-428f-a940-712a7415845b]],데이터 변화를 효과적으로 분석하기 위해 어떤 조치를 취했나요?,[시계열 데이터 처리 강화와 데이터 전처리 기법을 다양화하여 대폭 추가했습니다.]


In [12]:
# 위 내용을 엑설파일로 저장 
qa_df.to_excel('corpus_data/pandas_book_qa.xlsx', index=False)

In [13]:
qa_df.head()

,qid,retrieval_gt,query,generation_gt
0,98290e07-565e-4eda-b028-c8d28af7e589,[[f3b63111-8036-4f21-a629-667f3b416c72]],"""본문 소스 코드"" 자료의 등록일은 언제인가요?",[등록일은 2024년 6월 25일입니다.]
1,ff487150-3312-491d-8e4a-7869bc41e067,[[6efb506d-100e-4b81-be0e-cd39bbf237cb]],이 책에 추가된 주요 내용은 무엇인가요?,[시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.]
2,2b486627-da24-4d56-8abf-03b09ef9b642,[[70b74199-9865-4844-9355-8ed2de5febf8]],"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표일은 언제인가요?",[2024년 4월 24일입니다.]
3,680ab4e7-fa97-4167-98ec-6a874888c518,[[241cab43-3017-40bf-b703-12a6029a05ca]],[보도자료] 파이썬과 엑셀로 시작하는 딥러닝이 발표된 날짜는 언제인가요?,[이 보도자료는 2023년 7월 19일에 발표되었습니다.]
4,54b9798b-91aa-484c-8544-4600cbdabe38,[[039c58d5-6083-428f-a940-712a7415845b]],데이터 변화를 효과적으로 분석하기 위해 어떤 조치를 취했나요?,[시계열 데이터 처리 강화와 데이터 전처리 기법을 다양화하여 대폭 추가했습니다.]


In [14]:
# # 직접 수정한 QA 데이터를 불러옴
# qa_data = pd.read_excel('corpus_data/pandas_book_qa_final.xlsx')
# qa_data.head()

In [15]:
qa_data = pd.read_excel('corpus_data/pandas_book_qa.xlsx')
qa_data.head()

,qid,retrieval_gt,query,generation_gt
0,98290e07-565e-4eda-b028-c8d28af7e589,[['f3b63111-8036-4f21-a629-667f3b416c72']],"""본문 소스 코드"" 자료의 등록일은 언제인가요?",['등록일은 2024년 6월 25일입니다.']
1,ff487150-3312-491d-8e4a-7869bc41e067,[['6efb506d-100e-4b81-be0e-cd39bbf237cb']],이 책에 추가된 주요 내용은 무엇인가요?,['시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.']
2,2b486627-da24-4d56-8abf-03b09ef9b642,[['70b74199-9865-4844-9355-8ed2de5febf8']],"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표일은 언제인가요?",['2024년 4월 24일입니다.']
3,680ab4e7-fa97-4167-98ec-6a874888c518,[['241cab43-3017-40bf-b703-12a6029a05ca']],[보도자료] 파이썬과 엑셀로 시작하는 딥러닝이 발표된 날짜는 언제인가요?,['이 보도자료는 2023년 7월 19일에 발표되었습니다.']
4,54b9798b-91aa-484c-8544-4600cbdabe38,[['039c58d5-6083-428f-a940-712a7415845b']],데이터 변화를 효과적으로 분석하기 위해 어떤 조치를 취했나요?,['시계열 데이터 처리 강화와 데이터 전처리 기법을 다양화하여 대폭 추가했습니다.']


In [16]:
qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qid            36 non-null     object
 1   retrieval_gt   36 non-null     object
 2   query          36 non-null     object
 3   generation_gt  36 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


In [17]:
type(qa_data['retrieval_gt'][0])

str

In [18]:
type(qa_data['generation_gt'][0])

str

In [19]:
# str -> list 객체로 변환해주기 위해 

import ast
for col in ['retrieval_gt', 'generation_gt']:
    qa_data[col] = qa_data[col].apply(ast.literal_eval)

qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qid            36 non-null     object
 1   retrieval_gt   36 non-null     object
 2   query          36 non-null     object
 3   generation_gt  36 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


In [20]:
type(qa_data['retrieval_gt'][0])
# 리스트로 변함 

list

In [21]:
type(qa_data['generation_gt'][0])

list

## RAGAS - RAG Evaluation

In [22]:
# Chain 테스트
response = rag_chain.invoke("이 책의 출판사는 어디인가요?")
print(response)

이 책의 출판사は "정보문화社"입니다。


In [23]:
qa_data = qa_data[['query', 'generation_gt']]
qa_data.columns = ['question', 'ground_truth']
qa_data.head()

,question,ground_truth
0,"""본문 소스 코드"" 자료의 등록일은 언제인가요?",[등록일은 2024년 6월 25일입니다.]
1,이 책에 추가된 주요 내용은 무엇인가요?,[시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.]
2,"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표일은 언제인가요?",[2024년 4월 24일입니다.]
3,[보도자료] 파이썬과 엑셀로 시작하는 딥러닝이 발표된 날짜는 언제인가요?,[이 보도자료는 2023년 7월 19일에 발표되었습니다.]
4,데이터 변화를 효과적으로 분석하기 위해 어떤 조치를 취했나요?,[시계열 데이터 처리 강화와 데이터 전처리 기법을 다양화하여 대폭 추가했습니다.]


In [24]:
# 평가 준비, rag_chain(qwen2) 표 채우기 

qa_data['ground_truth'] = qa_data['ground_truth'].apply(lambda x: x[0] )
qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke(x) )
qa_data['contexts'] = qa_data['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)] )
qa_data.head()

C:\Users\kim_h\AppData\Local\Temp\ipykernel_15780\1755535353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['ground_truth'] = qa_data['ground_truth'].apply(lambda x: x[0] )
C:\Users\kim_h\AppData\Local\Temp\ipykernel_15780\1755535353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke(x) )
c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Ba

,question,ground_truth,answer,contexts
0,"""본문 소스 코드"" 자료의 등록일은 언제인가요?",등록일은 2024년 6월 25일입니다.,"본문에서는 ""본문 소스 코드""의 등록일에 대한 정보를 제공하지 않았습니다. 따라서 ...",[비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pa...
1,이 책에 추가된 주요 내용은 무엇인가요?,시계열 데이터의 효율적인 처리와 분석을 위한 다양한 기법이 추가되었습니다.,이 책에 추가된 주요 내용은 Pandas 2.0 버전에서 추가되거나 변경된 내용이며...,[비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pa...
2,"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표일은 언제인가요?",2024년 4월 24일입니다.,"""챗GPT 프롬프트 120% 질문 기술"" 보도자료의 발표일은 2024년 4월 24일...",[330\n\n[보도자료] 예제가 가득한 챗GPT 프롬프트 길라잡이\n운영자\n20...
3,[보도자료] 파이썬과 엑셀로 시작하는 딥러닝이 발표된 날짜는 언제인가요?,이 보도자료는 2023년 7월 19일에 발표되었습니다.,[보도자료] 파이썬과 엑셀로 시작하는 딥러닝이 발표된 날짜는 2023년 7월 19일...,"[지은이 | 오승환과학고, 서울대를 졸업하고, 중 국 C KGSB MBA, FRM(..."
4,데이터 변화를 효과적으로 분석하기 위해 어떤 조치를 취했나요?,시계열 데이터 처리 강화와 데이터 전처리 기법을 다양화하여 대폭 추가했습니다.,문서에 따르면 다음과 같이 데이터 변화를 효과적으로 분석하기 위한 조치들이 있습니다...,[대폭 추가해 시간에 따른 데이터 변화를 효과적으로 분석할 수 있도록 했다. 또한 ...


In [25]:
from datasets import Dataset 

dataset = Dataset.from_pandas(qa_data)

dataset[0]

{'question': '"본문 소스 코드" 자료의 등록일은 언제인가요?',
 'ground_truth': '등록일은 2024년 6월 25일입니다.',
 'answer': '본문에서는 "본문 소스 코드"의 등록일에 대한 정보를 제공하지 않았습니다. 따라서 이 질문에 대한 정확한 대답을 주어지는 것은 불가능합니다.',
 'contexts': ['비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하여 시계열 데이터 처리 등의 기법과',
  '데이터 과학자가 되기 위한 첫걸음!어려운 이론은 최소화하고, 예제 코드를 따라 하며 자연스럽게 사용법에 익숙해지도록 안내하는 데이터 분석 입문서다. 데이터 분석을 처음',
  '지은이 | 오승환과학고, 서울대를 졸업하고, 중 국 C KGSB MBA, FRM(미국 재무위험관리사) 자격을 보유하고 있다. 국내 주요 금융기관과 대기업에서 기업분석 및 전략기획 업무를 담당했다. IT 비전공자이지만 데이터 분석과 인공지능을 독학으로 익혔고, 현재 핀테크 스타트업 대표이자 인공지능 강사로 활발하게 활동하고 있다. 유튜브 <판다스 스튜디오>를 운영하며, 저서로는 『파이썬 딥러닝 텐서플로』 등이 있다.',
  '만큼 더욱 자세한 설명과 다양한 데이터 시각화, 데이터 전처리 기법을 담았으며, Pandas 2.0 버전에서 추가되거나 변경된 내용을 풍부하게 담아 데이터 분석에']}

In [26]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [27]:
from ragas import evaluate
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

# chatgpt 4 모델로 평가 하다가는 요금 폭탄 받을 수 있음 주의!! 
langchain_llm =  ChatOllama(model="qwen2")
langchain_llm2 =  ChatOllama(model="gemma2")
langchain_embeddings = OllamaEmbeddings(model="qwen2")
langchain_embeddings2 = OllamaEmbeddings(model="gemma2")

result = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm, 
    embeddings=langchain_embeddings,
    raise_exceptions=False,
)

result

Evaluating:   0%|          | 0/144 [00:00<?, ?it/s]

[06/29/24 22:15:43] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=450275;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py\output_parser.py]8;;\:]8;id=688593;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py#71\71]8;;\

[06/29/24 22:15:44] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=688189;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py\output_parser.py]8;;\:]8;id=743648;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py#71\71]8;;\

[06/29/24 22:16:52] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=549406;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py\output_parser.py]8;;\:]8;id=648499;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py#71\71]8;;\

                    WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=721444;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py\output_parser.py]8;;\:]8;id=126158;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py#71\71]8;;\

...

[06/29/24 22:29:01] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=953998;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py\output_parser.py]8;;\:]8;id=382718;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py#71\71]8;;\

[06/29/24 22:29:03] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=745191;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py\output_parser.py]8;;\:]8;id=187796;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py#71\71]8;;\

[06/29/24 22:30:58] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=441837;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py\output_parser.py]8;;\:]8;id=823502;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py#71\71]8;;\

[06/29/24 22:31:18] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=757871;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py\output_parser.py]8;;\:]8;id=161306;file://c:\Users\kim_h\anaconda3\envs\rag\Lib\site-packages\ragas\llms\output_parser.py#71\71]8;;\

{'faithfulness': 0.6364, 'answer_relevancy': 0.6405, 'context_recall': 1.0000, 'context_precision': 0.8526}

## Gemma2 모델을 사용한 RAG Chain

In [28]:
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chat Model
llm = ChatOllama(model="gemma2")

rag_chain2 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

qa_data2 = qa_data.copy()
qa_data2['answer'] = qa_data2['question'].apply(lambda x: rag_chain2.invoke(x) )
qa_data2.head()

In [ ]:
dataset2 = Dataset.from_pandas(qa_data2)

result2 = evaluate(
    dataset2,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm2, 
    embeddings=langchain_embeddings2,
    raise_exceptions=False,
)
# 4개의 평가지표에 대한 성능 값이 나옴 
result2

Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]

[06/29/24 05:40:24] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=567075;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py\output_parser.py]8;;\:]8;id=816044;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py#71\71]8;;\

[06/29/24 05:43:03] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=821002;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py\output_parser.py]8;;\:]8;id=28536;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py#71\71]8;;\

{'faithfulness': 0.9000, 'answer_relevancy': 0.3450, 'context_recall': 0.9000, 'context_precision': 0.9672}

## 모델 성능 비교

In [ ]:
df1 = pd.DataFrame(list(result.items()), columns=['Metric', 'Qwen2'])
df2 = pd.DataFrame(list(result2.items()), columns=['Metric', 'Gemma2'])

df_result = pd.merge(df1, df2, on='Metric')
df_result

,Metric,Qwen2,Gemma2
0,faithfulness,0.907576,0.900000
1,answer_relevancy,0.319209,0.345019
2,context_recall,0.900000,0.900000
3,context_precision,0.967172,0.967172
